In [2]:
(notebook_name)=
# 2.1 Building a Bayesian model for events
[2.1 Building a Bayesian model for events](https://www.bayesrulesbook.com/chapter-2.html#building-a-bayesian-model-for-events)

:::{post} March 22, 2022
:tags: python
:category: beginner
:author: Lode Nachtergaele
:::

SyntaxError: invalid syntax (1805177719.py, line 1)

<a href="https://github.com/pymc-devs/resources/blob/main/bayesrules/Section_2_1_Fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pyreadr
# !pip install wget
# !pip install pymc3
# !pip install arviz

In [7]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import wget
import pyreadr
import seaborn as sns

ModuleNotFoundError: No module named 'wget'

In [ ]:
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
RANDOM_SEED = 1301
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [ ]:
# !wget -O fake_news.rda https://github.com/bayes-rules/bayesrules/blob/master/data/fake_news.rda?raw=true

In [ ]:
fake_news_url = r'https://github.com/bayes-rules/bayesrules/blob/master/data/fake_news.rda?raw=true'  
wget.download(fake_news_url, 'fake_news.rda')
try:
    result = pyreadr.read_r('fake_news.rda')
except FileNotFoundError:
    print('Could not read R data file fake_news.rda')

In [ ]:
df = result['fake_news']

In [ ]:
df.head()

In [ ]:
pd.concat([df['type'].value_counts().rename('n'), df['type'].value_counts(normalize=True).rename('percent')], axis=1)

In [ ]:
pd.crosstab(df['title_has_excl'], df['type'], margins=True)

In [ ]:
article = pd.Series(["real", "fake"], dtype="category")

In [ ]:
prior = pd.Series([0.6, 0.4])

In [ ]:
df_sample = pd.DataFrame({'type': article, 'weights' : prior})

In [4]:
df_sample.sample(3, replace=True, random_state=1301, weights='weights')

NameError: name 'df_sample' is not defined

In [ ]:
article_sim = df_sample.sample(10_000, replace=True, random_state=84735, weights='weights')

In [ ]:
ax = article_sim['type'].value_counts().plot.bar(rot=0);
ax.set_xlabel('type');
ax.set(title='FIGURE 2.2: A bar plot of the fake vs real status of 10,000 simulated articles.');

In [ ]:
article_sim['type'].value_counts()

In [ ]:
article_sim['data_model'] = article_sim['type'].map({'fake': 0.2667, 'real':0.0222})
article_sim.head()

In [ ]:
N_fake = (article_sim['type']=='fake').sum() # The number of artices that are fake
N_real = (article_sim['type']=='real').sum() # The number of artices that are real
article_sim.loc[article_sim['type']=='fake', 'usage'] = np.random.choice(['no', 'yes'], size=N_fake, replace=True, p=[1-0.2667, 0.2667])
article_sim.loc[article_sim['type']=='real', 'usage'] = np.random.choice(['no', 'yes'], size=N_real, replace=True, p=[1-0.0222, 0.0222])

In [ ]:
pd.crosstab(article_sim['usage'], article_sim['type'], margins=True)

In [ ]:
fix, ax = plt.subplots(1, 2, figsize=(8, 4.5))
pd.crosstab(article_sim['type'], article_sim['usage']).plot(kind='bar', stacked=True, rot=0, ax=ax[0]);
sns.countplot(x='usage', data=article_sim, ax=ax[1]);
# article_sim['usage'].value_counts().plot.bar(rot=0, ax=ax[1]);
plt.suptitle('FIGURE 2.3: Bar plots of exclamation point usage, both within fake vs real news and overall.');
plt.subplots_adjust(top=0.15)
plt.tight_layout();

In [ ]:
# ax = article_sim['usage'].value_counts().plot.bar(rot=0);
# ax.set_xlabel('usage');

In [ ]:
# sns.catplot(x='usage', kind='count', data=article_sim);

In [ ]:
sns.catplot(x='usage', hue='type', data=article_sim, kind='count');
plt.title('FIGURE 2.4: Bar plots of real vs fake news, broken down by exclamation point usage.');

In [ ]:
%load_ext watermark
%watermark -n -u -v -iv -w -p wget, pyreadr, pandas, matplotlib, seaborn

In [ ]:
:::{include} ../page_footer.md
:::